In [6]:
import numpy as np
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array, array_to_img
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
from skimage import data, color
from PIL import Image
import cv2
from numpy import savez_compressed
from google.colab import files
from numpy import load
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from google.colab import drive
drive.mount('/content/gdrive')

filename = '/content/gdrive/My Drive/Research2/np_data/xkmnist.npz'
x = np.load(filename)['arr_0']

filename = '/content/gdrive/My Drive/Research2/np_data/ykmnist.npz'
y = np.load(filename)['arr_0']

print(x.shape)
print(y.shape)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
(70000, 56, 56, 1)
(70000, 1)


In [7]:
baseAccuracies = []
losses = []
balancedAccuracies = []
f1scores = []
precisions = []
recalls = []

kfold = KFold(n_splits=10, shuffle=True)
fold_no = 1
callback = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1)

for train, test in kfold.split(x, y):
  print("_________________________________________________________________________________________________________________________________")
  print("Fold number: " + str(fold_no))
  fold_no = fold_no+1
  print()

  xtrain = x[train]
  xtest = x[test]
  
  model = tf.keras.applications.ResNet152V2(include_top=True, weights=None, input_shape=(56, 56, 1), classes=10, classifier_activation="softmax",)
  model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(lr=0.0001), metrics=["sparse_categorical_accuracy"])
  model.fit(xtrain, y[train], verbose = 1, validation_data = (xtest, y[test]), epochs = 100, callbacks = [callback], batch_size = 32)

  test_loss, test_accuracy = model.evaluate(xtest, y[test])
  print("_________________________________________________________________________________________________________________________________")
  print("Test Accuracy:")
  print(test_accuracy)
  baseAccuracies.append(test_accuracy)
  print("_________________________________________________________________________________________________________________________________")
  print("Test Loss:")
  print(test_loss)
  losses.append(test_loss)
  print("_________________________________________________________________________________________________________________________________")
  preds = np.argmax(model.predict(xtest), axis=-1)
  preds = preds.reshape(len(xtest), 1)
  accs = []
  for cls in range(10):
    mask = (y[test] == cls)
    cls_acc = (preds == cls)[mask].mean() 
    accs.append(cls_acc)
  accs = np.mean(accs)
  print("Final balanced accuracy:")
  print(accs)
  balancedAccuracies.append(accs)
  print("_________________________________________________________________________________________________________________________________")
  f1scores.append(f1_score(y[test], preds, average = 'weighted'))
  precisions.append(precision_score(y[test], preds, average = 'weighted'))
  recalls.append(recall_score(y[test], preds, average = 'weighted'))
  print("*********************************************************************************************************************************")
  print()
  print()

_________________________________________________________________________________________________________________________________
Fold number: 1

Epoch 1/100
1969/1969 [==============================] - 204s 103ms/step - loss: 0.6004 - sparse_categorical_accuracy: 0.8049 - val_loss: 0.3786 - val_sparse_categorical_accuracy: 0.8803
Epoch 2/100
1969/1969 [==============================] - 202s 102ms/step - loss: 0.1840 - sparse_categorical_accuracy: 0.9430 - val_loss: 0.1259 - val_sparse_categorical_accuracy: 0.9629
Epoch 3/100
1969/1969 [==============================] - 202s 103ms/step - loss: 0.1057 - sparse_categorical_accuracy: 0.9665 - val_loss: 0.1232 - val_sparse_categorical_accuracy: 0.9644
Epoch 4/100
1969/1969 [==============================] - 201s 102ms/step - loss: 0.0734 - sparse_categorical_accuracy: 0.9771 - val_loss: 0.0899 - val_sparse_categorical_accuracy: 0.9727
Epoch 5/100
1969/1969 [==============================] - 201s 102ms/step - loss: 0.0571 - sparse_categoric

In [8]:
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(balancedAccuracies)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {losses[i]} - class-wise Accuracy: {balancedAccuracies[i]}% Accuracy - {baseAccuracies[i]} - Precision: {precisions[i]} - Recall: {recalls[i]} - F1: {f1scores[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(balancedAccuracies)} (+- {np.std(balancedAccuracies)})')
print(f'> Loss: {np.mean(losses)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.08650551736354828 - class-wise Accuracy: 0.9755203798112037% Accuracy - 0.9755714535713196 - Precision: 0.9760462063293195 - Recall: 0.9755714285714285 - F1: 0.9755798177275955
------------------------------------------------------------------------
> Fold 2 - Loss: 0.052710652351379395 - class-wise Accuracy: 0.986637154957692% Accuracy - 0.9865714311599731 - Precision: 0.986621189755759 - Recall: 0.9865714285714285 - F1: 0.9865731244851343
------------------------------------------------------------------------
> Fold 3 - Loss: 0.07572481036186218 - class-wise Accuracy: 0.9772912336970085% Accuracy - 0.9772857427597046 - Precision: 0.9775169367618598 - Recall: 0.9772857142857143 - F1: 0.9773034292234495
------------------------------------------------------------------------
> Fold 4 - Loss: 0.05755126848816